In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

import import_ipynb
# Импортируем полностью модули для передачи глобальных переменных
from utils import make_datasets
import models

# Построение гибридной системы

## Объединение моделей

In [2]:
def get_hybrid_recommendation(user_id, df_ratings, df_books, df_tags, df_book_tags, book_id=None, weights=None):
    """Функция объединяет прогнозы разных моделей и выводит список рекомендуемых книг.
    :param user_id: идентификатор пользователя
    :param book_id: идентификатор книги
    :param df_ratings: DataFrame с рейтингами
    :param df_books: DataFrame с информацией о книгах
    :param df_tags: DataFrame с тегами
    :param df_book_tags: DataFrame с привязанными тегами к книгам
    :param weights: весовые коэффициенты для моделей [популярность, похожесть, коллаборативная фильтрация]

    :return: список рекомендованных книг"""
    
    # Определим веса для каждой из рекомендаций
    if weights is None:
        weights = [2, 3, 4]
    
    # Получаем рекомендации книг по каждой модели в соответствии с весами
    popularity_rec = models.get_popularity_recommendation_ids(df_ratings, weights[0])
    content_rec = models.get_similar_books_ids(df_book_tags, df_tags, df_books, df_ratings, book_id, weights[1])
    collaborative_rec = models.get_recommendations_svd(user_id, df_ratings, weights[2])
    
    # Отбираем уникальные книги
    recommendations = (popularity_rec + content_rec + collaborative_rec)
    unique_recs = list(set(recommendations))
    
    return unique_recs

In [3]:
def get_user_type(user_id, df_users, threshold=None):
    """Классификация типа пользователя: новый или активный.
Основывается на количестве прочтённых книг и средней оценке.
    :param user_id: идентификатор пользователя
    :param df_users: DataFrame c информацией о пользователях

    :return: строка ('new' или 'active')
    """
    # Находим характеристики пользователя
    avg_user_rating = df_users[df_users['user_id'] == user_id]['avg_user_rating'].iloc[0]
    num_user_ratings = df_users[df_users['user_id'] == user_id]['num_user_ratings'].iloc[0]
    user_activity = df_users[df_users['user_id'] == user_id]['user_activity'].iloc[0]
    
    # Определяем погоровые значения
    if threshold is None:
        threshold = [3.5, 10, 0.1]

    th_avg_user_rating, th_num_user_ratings, th_user_activity = threshold

    if avg_user_rating >= th_avg_user_rating and \
      (num_user_ratings >= th_num_user_ratings or user_activity >= th_user_activity):
        user_type = 'active'
    else:
        user_type = 'new'

    return user_type

In [14]:
def get_combined_recomendation_by_user_type(user_id, df_ratings, df_books, df_tags, df_book_tags, df_users, book_id=None):
    """Комбинирование персонализированных и популярных рекомендаций на основе типа пользователя.
Для новых пользователей приоритет отдаётся популярным книгам, активным пользователям предлагаются 
персональные рекомендации.
    :param user_id: идентификатор пользователя
    :param book_id: идентификатор книги
    :param df_ratings: DataFrame с рейтингами
    :param df_books: DataFrame с информацией о книгах
    :param df_tags: DataFrame с тегами
    :param df_book_tags: DataFrame с привязанными тегами к книгам

    :return: список рекомендованных книг
    """
    user_type = get_user_type(user_id, df_users)

    # Новым пользователям предлагаются деперсонализированные популярные книги
    # Активным пользователям - гибридные взвешенные рекомендации
    if user_type == 'new':
        combined_recs = get_popularity_recommendation_ids(df_ratings)
    elif user_type == 'active':
        combined_recs = get_hybrid_recommendation(user_id, df_ratings, df_books, df_tags, df_book_tags, book_id)

    return combined_recs

# Запуск

In [15]:
# Определяем запуск из-под скрипта:
if __name__ == '__main__':
    data_path = os.path.abspath('../data')
    # book_id = 6621
    user_id = 315

    # Загружаем даныне
    print('Загружаем данные...')
    df_ratings, df_books, df_tags, df_book_tags, df_users = make_datasets(data_path)

    # Создаем матрицу взаимодействий
    print('Создаем матрицу взаимодействий...')
    # df_interaction_matrix = generate_user_book_similarity_matrix(df_ratings, df_books, df_book_tags, df_tags)
    
    # Обучаем модель SVD
    print('Обучаем модель SVD...')
    # models.best_params = prepare_model_svd_mode(df_ratings, 'increment')
    models.best_params = {'n_factors': 100, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.1}

    # Получаем гибридные рекомендации
    print('Получаем гибридные рекомендации...')
    unique_recs = get_hybrid_recommendation(user_id, df_ratings, df_books, df_tags, df_book_tags)
    print(unique_recs)

    # Получаем тип пользователя
    print('Получаем тип пользователя...')
    user_type = get_user_type(user_id, df_users)
    print(user_type)

    # Получаем комбинированные рекомендации в зависимости от типа пользователя
    print('Получаем комбинированные рекомендации в зависимости от типа пользователя...')
    combined_recs = get_combined_recomendation_by_user_type(user_id, df_ratings, df_books, df_tags, df_book_tags, df_users)
    print(combined_recs)

Загружаем данные...
Создаем матрицу взаимодействий...
Обучаем модель SVD...
Получаем гибридные рекомендации...
[2850, 3491, 2244, 3080, 4557, 9842, 8946]
Получаем тип пользователя...
active
Получаем комбинированные рекомендации в зависимости от типа пользователя...
[2850, 1380, 3080, 6920, 4557, 8946, 5207]
